In [1]:
#sigh - https://www.reddit.com/r/adventofcode/comments/ee0rqi/2019_day_22_solutions/

import re

with open('22.txt') as file:
    input = file.read()

rules = input.split('\n')

In [2]:
%%time

def shuffle(deck, rules):
    for s in rules:
        if s == 'deal into new stack':
            deck.reverse()
            continue
        match = re.match(r'cut (-?\d+)', s)
        if match != None:
            n = int(match.group(1))
            deck = deck[n:]+deck[:n]
            continue
        match = re.match(r'deal with increment (\d+)', s)
        if match != None:
            n = int(match.group(1))
            ndeck = [0]*len(deck)
            for i in range(len(deck)):
                ndeck[(i*n)%len(deck)] = deck[i]
            deck = ndeck
            continue
        raise Exception('unknown rule', s)
    return deck

deck = shuffle(list(range(10007)), rules)
deck.index(2019)

CPU times: user 51.6 ms, sys: 0 ns, total: 51.6 ms
Wall time: 51 ms


8379

In [3]:
# convert rules to linear polynomial.
# (g∘f)(x) = g(f(x))
def parse(L, rules):
    a,b = 1,0
    for s in rules[::-1]:
        if s == 'deal into new stack':
            a = -a
            b = L-b-1
            continue
        if s.startswith('cut'):
            n = int(s.split(' ')[1])
            b = (b+n)%L
            continue
        if s.startswith('deal with increment'):
            n = int(s.split(' ')[3])
            z = pow(n,L-2,L) # == modinv(n,L)
            a = a*z % L
            b = b*z % L
            continue
        raise Exception('unknown rule', s)
    return a,b

# modpow the polynomial: (ax+b)^m % n
# f(x) = ax+b
# g(x) = cx+d
# f^2(x) = a(ax+b)+b = aax + ab+b
# f(g(x)) = a(cx+d)+b = acx + ad+b
def polypow(a,b,m,n):
    if m==0:
        return 1,0
    if m%2==0:
        return polypow(a*a%n, (a*b+b)%n, m//2, n)
    else:
        c,d = polypow(a,b,m-1,n)
        return a*c%n, (a*d+b)%n

def shuffle2(L, N, pos, rules):
    a,b = parse(L,rules)
    #a,b = polypow(a,b,N,L)
    return (pos*a+b)%L

In [4]:
%%time

# test it out
pos = 2019
L = 10007
N = 10
deck = list(range(L))

print(shuffle2(L,N,pos,rules))

5343
6194
CPU times: user 496 ms, sys: 0 ns, total: 496 ms
Wall time: 495 ms


In [5]:
%%time

L = 119315717514047
N = 101741582076661
shuffle2(L,N,2020,rules)

CPU times: user 391 µs, sys: 0 ns, total: 391 µs
Wall time: 394 µs


25715231120204